<a href="https://colab.research.google.com/github/shemi024/ANN/blob/main/Breastcanser_ANN_imbalanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/breast-cancer_csv.csv')
df.sample(10)

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
44,30-39,premeno,15-19,0-2,no,1,left,left_low,no,no-recurrence-events
268,60-69,ge40,30-34,0-2,no,1,left,left_up,no,no-recurrence-events
236,30-39,lt40,15-19,0-2,no,3,right,left_up,no,no-recurrence-events
84,30-39,premeno,0-4,0-2,no,2,right,central,no,no-recurrence-events
181,60-69,ge40,10-14,6-8,yes,3,left,left_up,yes,recurrence-events
4,40-49,premeno,30-34,3-5,yes,2,left,right_up,no,recurrence-events
109,40-49,premeno,20-24,0-2,no,2,left,right_low,no,no-recurrence-events
88,50-59,ge40,20-24,0-2,no,1,right,left_low,no,no-recurrence-events
45,50-59,premeno,30-34,0-2,no,3,left,right_up,no,recurrence-events
213,50-59,ge40,25-29,0-2,no,1,left,right_low,no,no-recurrence-events


In [ ]:
df.isna().any()

age            False
menopause      False
tumor-size     False
inv-nodes      False
node-caps       True
deg-malig      False
breast         False
breast-quad     True
irradiat       False
Class          False
dtype: bool

In [ ]:
df.dropna(inplace=True)
df.isna().any()

age            False
menopause      False
tumor-size     False
inv-nodes      False
node-caps      False
deg-malig      False
breast         False
breast-quad    False
irradiat       False
Class          False
dtype: bool

In [ ]:
df.dtypes


age            object
menopause      object
tumor-size     object
inv-nodes      object
node-caps      object
deg-malig       int64
breast         object
breast-quad    object
irradiat       object
Class          object
dtype: object

In [ ]:
def print_unique_col_values(df):
  for column in df:
    if df[column].dtypes=='object':
      print(f'{column}: {df[column].unique()}')

In [ ]:
print_unique_col_values(df)

age: ['40-49' '50-59' '60-69' '30-39' '70-79' '20-29']
menopause: ['premeno' 'ge40' 'lt40']
tumor-size: ['15-19' '35-39' '30-34' '25-29' '40-44' '10-14' '0-4' '20-24' '45-49'
 '50-54' '5-9']
inv-nodes: ['0-2' '3-5' '15-17' '6-8' '9-11' '24-26' '12-14']
node-caps: ['yes' 'no']
breast: ['right' 'left']
breast-quad: ['left_up' 'central' 'left_low' 'right_up' 'right_low']
irradiat: ['no' 'yes']
Class: ['recurrence-events' 'no-recurrence-events']


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
for i in df:
  if df[i].dtype==np.dtype('O'):
    df[i]=le.fit_transform(df[i])

In [ ]:
df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,2,2,2,0,1,3,1,2,0,1
1,3,0,2,0,0,1,1,0,0,0
2,3,0,6,0,0,2,0,1,0,1
3,2,2,6,0,1,3,1,1,1,0
4,2,2,5,4,1,2,0,4,0,1


In [ ]:
X=df.drop(['Class'],axis=1)
y=df['Class']


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=10,stratify=y)

In [ ]:
X_train.shape


(221, 9)

In [ ]:
X_test.shape

(56, 9)

In [ ]:
# Build a model (ANN) in tensorflow/keras
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
def ANN(X_train,y_train,X_test,y_test,loss):
  model = keras.Sequential([
                          keras.layers.Dense(9, input_shape=(9,), activation='relu'),
                          keras.layers.Dense(5, activation='relu'),
                          keras.layers.Dense(1, activation='sigmoid'),
  ])
  model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'])
  
  
  model.fit(X_train,y_train, epochs=100)
 
  # model.fit(X_train,y_train, epochs=100, class_weight=weights)

  print(model.evaluate(X_test,y_test))

  y_preds = model.predict(X_test)
  y_preds = np.round(y_preds)

  print("classification report: \n", classification_report(y_test,y_preds))

  return y_preds

In [ ]:
y_preds= ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
7/7 [==============================] - 1s 3ms/step - loss: 0.6891 - accuracy: 0.6968
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6582 - accuracy: 0.6787
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6445 - accuracy: 0.6652
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6424 - accuracy: 0.6878
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6379 - accuracy: 0.6968
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6330 - accuracy: 0.7149
Epoch 7/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6273 - accuracy: 0.7104
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6215 - accuracy: 0.7104
Epoch 9/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6148 - accuracy: 0.7104
Epoch 10/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6075 - accuracy: 0.7149
Epoch 11/100
7/7 [===========

In [ ]:
# method 1: undersampling
# class count
# class1 include 1033 samples and 0 class include 374 samples so there is data imbalance



count_class_0,count_class_1 = df.Class.value_counts()

# divide by class

df_class_0 = df[df['Class'] == 0]
df_class_1 = df[df['Class'] == 1]

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)


print("Random under- sampling:")
print(df_test_under.Class.value_counts())

Random under- sampling:
0    81
1    81
Name: Class, dtype: int64


In [ ]:
X=df.drop(['Class'],axis=1)
y=df['Class']


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=10,stratify=y)


In [ ]:
y_train.value_counts()

0    156
1     65
Name: Class, dtype: int64

In [ ]:
y_preds= ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6365 - accuracy: 0.6742
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6225 - accuracy: 0.7059
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6118 - accuracy: 0.7059
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6044 - accuracy: 0.7104
Epoch 5/100
7/7 [==============================] - 0s 2ms/step - loss: 0.5994 - accuracy: 0.7149
Epoch 6/100
7/7 [==============================] - 0s 2ms/step - loss: 0.5949 - accuracy: 0.7195
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 0.5911 - accuracy: 0.7195
Epoch 8/100
7/7 [==============================] - 0s 3ms/step - loss: 0.5876 - accuracy: 0.7195
Epoch 9/100
7/7 [==============================] - 0s 2ms/step - loss: 0.5849 - accuracy: 0.7195
Epoch 10/100
7/7 [==============================] - 0s 2ms/step - loss: 0.5826 - accuracy: 0.7240
Epoch 11/100
7/7 [===========

In [ ]:
# Method2 = oversampling
# oversample 1-class and concat the dataframes of both clases

df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0,df_class_1_over],axis=0)

print("Random over- sampling:")
print(df_test_over.Class.value_counts())

Random over- sampling:
0    196
1    196
Name: Class, dtype: int64


In [ ]:
X=df.drop(['Class'],axis=1)
y=df['Class']


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=10,stratify=y)

In [ ]:
y_preds= ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')


Epoch 1/100
7/7 [==============================] - 0s 2ms/step - loss: 1.6176 - accuracy: 0.2941
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 1.4492 - accuracy: 0.3213
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 1.2901 - accuracy: 0.3484
Epoch 4/100
7/7 [==============================] - 0s 2ms/step - loss: 1.1632 - accuracy: 0.3529
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 1.0575 - accuracy: 0.3801
Epoch 6/100
7/7 [==============================] - 0s 2ms/step - loss: 0.9766 - accuracy: 0.3891
Epoch 7/100
7/7 [==============================] - 0s 2ms/step - loss: 0.9054 - accuracy: 0.4570
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 0.8574 - accuracy: 0.4661
Epoch 9/100
7/7 [==============================] - 0s 3ms/step - loss: 0.8157 - accuracy: 0.5068
Epoch 10/100
7/7 [==============================] - 0s 2ms/step - loss: 0.7840 - accuracy: 0.5113
Epoch 11/100
7/7 [===========

In [ ]:
X=df.drop(['Class'],axis=1)
y=df['Class']


# smote

In [ ]:
from imblearn.over_sampling import SMOTE

smote=SMOTE(sampling_strategy='minority')
X_sm,y_sm=smote.fit_resample(X,y)

y_sm.value_counts()

1    196
0    196
Name: Class, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=15, stratify=y)

In [ ]:
y_preds= ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6518 - accuracy: 0.6923
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6445 - accuracy: 0.7059
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6401 - accuracy: 0.7059
Epoch 4/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6366 - accuracy: 0.7059
Epoch 5/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6334 - accuracy: 0.7059
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6303 - accuracy: 0.7059
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6282 - accuracy: 0.7059
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6259 - accuracy: 0.7059
Epoch 9/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6234 - accuracy: 0.7059
Epoch 10/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6211 - accuracy: 0.7059
Epoch 11/100
7/7 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_train.value_counts()

0    156
1     65
Name: Class, dtype: int64

In [ ]:
y_preds= ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
7/7 [==============================] - 0s 3ms/step - loss: 0.9003 - accuracy: 0.2941
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 0.8472 - accuracy: 0.2986
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 0.8027 - accuracy: 0.3077
Epoch 4/100
7/7 [==============================] - 0s 3ms/step - loss: 0.7647 - accuracy: 0.3484
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 0.7352 - accuracy: 0.3620
Epoch 6/100
7/7 [==============================] - 0s 2ms/step - loss: 0.7117 - accuracy: 0.4842
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6887 - accuracy: 0.5294
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6695 - accuracy: 0.6199
Epoch 9/100
7/7 [==============================] - 0s 3ms/step - loss: 0.6535 - accuracy: 0.6833
Epoch 10/100
7/7 [==============================] - 0s 2ms/step - loss: 0.6408 - accuracy: 0.6652
Epoch 11/100
7/7 [===========

In [ ]:
# method4: use of ensemble with undersampling

df.Class.value_counts()

0    196
1     81
Name: Class, dtype: int64

In [ ]:
X = df.drop('Class',axis='columns')
y = df['Class']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2,random_state=15, stratify=y)


In [ ]:
df1=X_train.copy()
df1['Class']=y_train

In [ ]:
df1.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
237,2,2,5,1,1,3,0,2,1,1
167,2,2,3,0,0,2,0,0,0,0
100,2,2,3,4,1,2,0,1,1,1
118,2,2,6,6,1,2,1,2,1,0
147,4,0,1,0,0,1,0,1,0,0


In [ ]:
df1_class0 = df1[df1.Class == 0]
df1_class1 = df1[df1.Class == 1]

In [ ]:
def get_train_batch(df_majority, df_minority, start,end):
  df_train=pd.concat([df_majority[start:end], df_minority], axis=0)


  X_train=df_train.drop('Class',axis='columns')
  y_train=df_train.Class
  return X_train,y_train

In [ ]:
X_train, y_train =get_train_batch(df1_class0, df1_class1, 0, 1495)

y_pred1=ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
7/7 [==============================] - 0s 2ms/step - loss: 1.4817 - accuracy: 0.2941
Epoch 2/100
7/7 [==============================] - 0s 3ms/step - loss: 1.3352 - accuracy: 0.2941
Epoch 3/100
7/7 [==============================] - 0s 3ms/step - loss: 1.2080 - accuracy: 0.2941
Epoch 4/100
7/7 [==============================] - 0s 2ms/step - loss: 1.0973 - accuracy: 0.2941
Epoch 5/100
7/7 [==============================] - 0s 2ms/step - loss: 1.0063 - accuracy: 0.2986
Epoch 6/100
7/7 [==============================] - 0s 2ms/step - loss: 0.9248 - accuracy: 0.3032
Epoch 7/100
7/7 [==============================] - 0s 2ms/step - loss: 0.8670 - accuracy: 0.3213
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 0.8156 - accuracy: 0.3394
Epoch 9/100
7/7 [==============================] - 0s 3ms/step - loss: 0.7726 - accuracy: 0.3937
Epoch 10/100
7/7 [==============================] - 0s 2ms/step - loss: 0.7420 - accuracy: 0.4661
Epoch 11/100
7/7 [===========

In [ ]:
X_train, y_train =get_train_batch(df1_class0, df1_class1, 1495,2990)

y_pred2=ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
3/3 [==============================] - 0s 4ms/step - loss: 0.4264 - accuracy: 0.9385
Epoch 2/100
3/3 [==============================] - 0s 4ms/step - loss: 0.3827 - accuracy: 0.9692
Epoch 3/100
3/3 [==============================] - 0s 4ms/step - loss: 0.3437 - accuracy: 0.9846
Epoch 4/100
3/3 [==============================] - 0s 5ms/step - loss: 0.3100 - accuracy: 0.9846
Epoch 5/100
3/3 [==============================] - 0s 5ms/step - loss: 0.2796 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 5ms/step - loss: 0.2520 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 4ms/step - loss: 0.2277 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 6ms/step - loss: 0.2066 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.1885 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 4ms/step - loss: 0.1722 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
X_train, y_train =get_train_batch(df1_class0, df1_class1, 2990,4130)

y_pred3=ANN(X_train,y_train,X_test,y_test, 'binary_crossentropy')

Epoch 1/100
3/3 [==============================] - 0s 5ms/step - loss: 0.8026 - accuracy: 0.5077
Epoch 2/100
3/3 [==============================] - 0s 5ms/step - loss: 0.7696 - accuracy: 0.5538
Epoch 3/100
3/3 [==============================] - 0s 4ms/step - loss: 0.7385 - accuracy: 0.5538
Epoch 4/100
3/3 [==============================] - 0s 4ms/step - loss: 0.7119 - accuracy: 0.5846
Epoch 5/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6858 - accuracy: 0.5846
Epoch 6/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6607 - accuracy: 0.6000
Epoch 7/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6361 - accuracy: 0.6000
Epoch 8/100
3/3 [==============================] - 0s 5ms/step - loss: 0.6131 - accuracy: 0.6154
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5895 - accuracy: 0.6154
Epoch 10/100
3/3 [==============================] - 0s 4ms/step - loss: 0.5683 - accuracy: 0.6308
Epoch 11/100
3/3 [===========

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred_final=y_pred1.copy()
for i in range(len(y_pred1)):
  n_ones=y_pred1[i]+y_pred2[i]+y_pred3[i]
  if n_ones>1:
    y_pred_final[i]=1
  else:
    y_pred_final[i]=0
    

In [ ]:
cl_rep=classification_report(y_test,y_pred_final)
print(cl_rep)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        40
           1       0.29      1.00      0.44        16

    accuracy                           0.29        56
   macro avg       0.14      0.50      0.22        56
weighted avg       0.08      0.29      0.13        56



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
